# Reviewing Historical IDPs by Cyclone

In [4]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely.geometry import LineString, Point, MultiLineString
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import seaborn as sns
from src.datasources import codab, rsmc
from src import constants

c:\Users\pauni\Desktop\Work\OCHA\GitHub\ds-aa-mdg-storms\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cyclone_db = (
    Path(constants.AA_DATA_DIR)
    / "public"
    / "raw"
    / "mdg"
    / "cyclone_database"
    / "Synthèse dégâts Climatiques 1997 - 2024 (OCHA).xlsx"
)
cyclone_db = pd.read_excel(cyclone_db, sheet_name="BDD").dropna(how="all")
cyclone_db["Nom"] = cyclone_db["Nom"].str.upper()

In [3]:
cerf_emdat_df = pd.read_csv(
    Path(constants.AA_DATA_DIR)
    / "public/exploration/mdg/cerf_emdat_bngrc_data.csv"
)

In [30]:
grouped = cyclone_db.groupby("Nom")["Deplacees"].sum()


# Define a function to classify based on the sum of 'Deplacees'
def classify_scenario(deplacees_sum):
    if 1000 <= deplacees_sum <= 10000:
        return "Scenario 1"
    elif 10000 < deplacees_sum <= 25000:
        return "Scenario 2"
    elif deplacees_sum > 25000:
        return "Scenario 3"
    else:
        return "No Scenario"


# Apply the classification
grouped_scenario = grouped.apply(classify_scenario)

# Combine the sum and classification into a new DataFrame
grouped_df = pd.DataFrame(
    {"Deplacees Sum": grouped, "Scenario": grouped_scenario}
)

In [31]:
merged_df = pd.merge(cerf_emdat_df, grouped_df, on="Nom", how="inner")
merged_df.columns

Index(['Nom', 'Total Affected - EMDAT', 'Sinistres', 'CERF Allocations',
       'Deplacees Sum', 'Scenario'],
      dtype='object')

In [32]:
# Sort the grouped DataFrame by 'Deplacees Sum' in descending order
grouped_sorted = merged_df.sort_values(
    by=["Scenario", "Total Affected - EMDAT", "CERF Allocations"],
    ascending=[
        False,
        False,
        False,
    ],  # Adjust order: descending for Scenario and Total Affected, ascending for Deplacees
)


# Define functions for highlighting and coloring bars
def color_bar_total_affected(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, orange {val/grouped_sorted["Total Affected - EMDAT"].max()*100}%, transparent {val/grouped_sorted["Total Affected - EMDAT"].max()*100}%);'
    return ""


def color_bar_sinistres(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, #FFD700 {val/grouped_sorted["Sinistres"].max()*100}%, transparent {val/grouped_sorted["Sinistres"].max()*100}%);'
    return ""


def color_bar_cerf(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, green {val/grouped_sorted["CERF Allocations"].max()*100}%, transparent {val/grouped_sorted["CERF Allocations"].max()*100}%);'
    return ""


def color_bar_deplacees_sum(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, lightblue {val/grouped_sorted["Deplacees Sum"].max()*100}%, transparent {val/grouped_sorted["Deplacees Sum"].max()*100}%);'
    return ""


def highlight_scenario(val):
    if val == "Scenario 1":
        return "background-color: lightblue;"
    elif val == "Scenario 2":
        return "background-color: lightgreen;"
    elif val == "Scenario 3":
        return "background-color: lightcoral;"
    return ""


# Apply styling
styled_grouped_df = (
    grouped_sorted.style.map(
        color_bar_total_affected, subset=["Total Affected - EMDAT"]
    )
    .map(color_bar_sinistres, subset=["Sinistres"])
    .map(color_bar_cerf, subset=["CERF Allocations"])
    .map(color_bar_deplacees_sum, subset=["Deplacees Sum"])
    .applymap(highlight_scenario, subset=["Scenario"])
    .format(
        {
            "Total Affected - EMDAT": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),
            "Sinistres": lambda x: f"{int(x):,}" if pd.notna(x) else "",
            "CERF Allocations": lambda x: f"{int(x):,}" if pd.notna(x) else "",
            "Deplacees Sum": lambda x: f"{int(x):,}" if pd.notna(x) else "",
        }
    )
    .set_table_styles(
        {"": [{"selector": "table", "props": "background-color: white;"}]}
    )
)

# Display the styled DataFrame
styled_grouped_df

C:\Users\pauni\AppData\Local\Temp\ipykernel_3548\2764567003.py:55: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_scenario, subset=["Scenario"])


,Nom,Total Affected - EMDAT,Sinistres,CERF Allocations,Deplacees Sum,Scenario
15,IVAN,"524,153","487,146","4,625,583","191,182",Scenario 3
32,ENAWO,"434,253","437,443","4,999,601","247,219",Scenario 3
45,FREDDY,"299,000","189,352","7,033,283","72,773",Scenario 3
11,INDLALA,"215,198","1,740,911","1,230,903","42,112",Scenario 3
40,EMNATI,"169,000","172,178","1,470,268","46,596",Scenario 3
38,BATSIRAI,"112,115","166,671","4,476,918","63,625",Scenario 3
44,CHENESO,"90,870","90,519",,"53,618",Scenario 3
7,GAFILO,,"860,437",,"305,812",Scenario 3
41,ANA,"131,555",,,"12,051",Scenario 2
47,GAMANE,"89,469","89,465","3,000,000","22,688",Scenario 2


In [39]:
# Step 1: Group by 'Scenario' and count occurrences
scenario_counts = (
    grouped_df.groupby("Scenario").size().reset_index(name="Count")
)

# Step 2: Calculate return periods (assuming 28 years)
scenario_counts["Return Period (Years)"] = 28 / scenario_counts["Count"]

# Step 3: Add cumulative scenarios (e.g., 'Scenario 3', '2 and 3', '1, 2 and 3')
# Initialize counts for the cumulative scenarios
total_counts = {
    "Scenario 3": scenario_counts.loc[
        scenario_counts["Scenario"] == "Scenario 3", "Count"
    ].sum(),
    "Scenario 2 and 3": scenario_counts.loc[
        scenario_counts["Scenario"].isin(["Scenario 2", "Scenario 3"]), "Count"
    ].sum(),
    "Scenario 1, 2 and 3": scenario_counts["Count"].sum(),
}

# Create a DataFrame for cumulative scenarios
cumulative_df = pd.DataFrame.from_dict(
    {
        "Scenario": total_counts.keys(),
        "Count": total_counts.values(),
        "Return Period (Years)": [
            28 / count for count in total_counts.values()
        ],
    }
)

# Step 4: Combine the grouped and cumulative results
final_scenario_table = pd.concat(
    [scenario_counts, cumulative_df], ignore_index=True
)

# Step 5: Sort the final table for clarity
final_scenario_table = final_scenario_table.sort_values(by="Scenario")

# Display the final table
final_scenario_table

,Scenario,Count,Return Period (Years)
0,No Scenario,41,0.682927
1,Scenario 1,9,3.111111
6,"Scenario 1, 2 and 3",64,0.437500
2,Scenario 2,5,5.600000
5,Scenario 2 and 3,14,2.000000
3,Scenario 3,9,3.111111
4,Scenario 3,9,3.111111
